# Finding the Best Time to Post on Hacker News to Get the Most Comments

## Introduction

Our main task in this project is to analyze **Hacker News** posts. We want to find out which posts are more likely to get the most comments. We also want to understand if the post creation time affects the number of comments they receive. As for some extra, we are going to do the same research for the points the posts receive.

There are two types of posts on **Hacker News**. One of them is **Ask HN** where users can ask the **Hacker News** community a specific question. And the second one is **Show HN** where users show the Hacker News community a project, product, or just something interesting. We are going to focusing on these two types of posts to do our research.

In our research, we learned that **Ask HN** gets the most comments. The creation time of the post also affects the number of comments per the post. The following will show the steps we used to achieve our goal.

## Open and Read Through the File

We are starting from importing modules that will help us in this project.

In [1]:
from csv import reader
import datetime

Now we are ready to open and read our dataset. We are using [this](https://www.kaggle.com/datasets/hacker-news/hacker-news-posts) dataset from **Kaggle.com**. You can download it directly from [this link](https://www.kaggle.com/datasets/hacker-news/hacker-news-posts/download?datasetVersionNumber=1).

In [2]:
opened_file = open('C:/Users/feelingcxld Study/my_datasets/HN_posts_year_to_Sep_26_2016.csv',
                encoding = 'utf8')
read_file = reader(opened_file)

hn = list(read_file)

header = hn[0]
hn = hn[1:]

Here's the column names from our dataset. Let's take a closer look at them.

In [3]:
print(header)

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


* `id`: the unique Identifier from **Hacker News** for the post
* `title`: the title of the post
* `url`: the URL that the posts links to, if the post has a URL
* `num_points`: the number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
* `num_comments`: the number of comments on the post
* `author`: the username of the person who submitted the post
* `created_at`: the date and time of the post's submission

In [4]:
print(hn[:4 + 1])

[['12579008', 'You have two days to comment if you want stem cells to be classified as your own', 'http://www.regulations.gov/document?D=FDA-2015-D-3719-0018', '1', '0', 'altstar', '9/26/2016 3:26'], ['12579005', 'SQLAR  the SQLite Archiver', 'https://www.sqlite.org/sqlar/doc/trunk/README.md', '1', '0', 'blacksqr', '9/26/2016 3:24'], ['12578997', 'What if we just printed a flatscreen television on the side of our boxes?', 'https://medium.com/vanmoof/our-secrets-out-f21c1f03fdc8#.ietxmez43', '1', '0', 'pavel_lishin', '9/26/2016 3:19'], ['12578989', 'algorithmic music', 'http://cacm.acm.org/magazines/2011/7/109891-algorithmic-composition/fulltext', '1', '0', 'poindontcare', '9/26/2016 3:16'], ['12578979', 'How the Data Vault Enables the Next-Gen Data Warehouse and Data Lake', 'https://www.talend.com/blog/2016/05/12/talend-and-Â\x93the-data-vaultÂ\x94', '1', '0', 'markgainor1', '9/26/2016 3:14']]


And this is some rows from our dataset.

## Extracting Ask HN and Show HN Posts

First, we will separate the **Ask HN** and **Show HN** posts. 

In [5]:
ask_posts = list()
show_posts = list()
other_posts = list()

# sorting posts by it's title
for row in hn:
    title = row[1].lower()
    
    if title.startswith('ask hn'):
        ask_posts.append(row)
        
    elif title.startswith('show hn'):
        show_posts.append(row)
    
    else:
        other_posts.append(row)
        
print('Number of \'Ask HN\' posts:', len(ask_posts))
print('Number of \'Show HN\' posts:', len(show_posts))
print('Number of other posts:', len(other_posts))

Number of 'Ask HN' posts: 9139
Number of 'Show HN' posts: 10158
Number of other posts: 273822


There are 9,139 **Ask HN** posts and 10,158 **Show HN** posts. They are quite equal, but **Show HN** posts are slightly more.

## Counting the Average Number of Comments per Post on Ask HN and Show HN

Let's find out how many comments both **Ask HN** and **Show HN** have in total.

In [6]:
# counting number of comments or points in show hn, ask hn or other posts
def count_by_post(title, row_index):    # title: ask_posts, show_posts or other_posts
                                        # row_index: 3 or 4
    total = 0
    
    for row in title:
        num = int(row[row_index])
        total += num
        
    return total

In [7]:
total_ask_comments = count_by_post(ask_posts, 4)
total_show_comments = count_by_post(show_posts, 4)
    
print('There are {c:,} Ask HN comments in total'.format(c = total_ask_comments))
print('There are {c:,} Show HN comments in total'.format(c = total_show_comments))

There are 94,986 Ask HN comments in total
There are 49,633 Show HN comments in total


We can see that the **Ask HN** posts have 94,986 comments in total and **Show HN** have only 49,633 comments in total. Now let's find the average number of comments per post for both **Ask HN** and **Show HN** posts.

In [8]:
# counting average number of comments or points for ask hn, show hn or other posts
def count_avg(title, total):    # title: ask_posts, show_posts or other_posts,
                                # total: total number of comments/points
    avg = total / len(title)
    return avg

In [9]:
avg_ask_comments = count_avg(ask_posts, total_ask_comments)
avg_show_comments = count_avg(show_posts, total_show_comments)

print('Average number of comments per \'Ask HN\' post:', round(avg_ask_comments, 2))
print('Average number of comments per \'Show HN\' post:', round(avg_show_comments, 2))

Average number of comments per 'Ask HN' post: 10.39
Average number of comments per 'Show HN' post: 4.89


On average, for the **Ask HN** we have 10.39 comments per posts and for the **Show HN** it's only 4.89 comments per post. This shows us that people are more willing to comment on **Ask HN** posts than **Show HN**. Based on this fact, for our next steps we will analyze only **Ask HN** posts.

## Looking for the Average Number of Comments by Hour for Ask HN Posts

### Counting the Number of Posts and Comments by Hour for Ask HN Posts

The next step is to find out whether or not the creation time of the post affects the number of comments received. We start by counting the number of posts and comments by hour for **Ask HN** posts. Doing so will help us to calculate the average number of comments per post by hour.

In [10]:
# isolating post creation time and number of comments or points for ask hn, show hn or other posts
def iso_data(title, row_index):    # title: ask_posts, show_posts or other_posts,
                                  # row_index: 3 or 4
    result = list()
    
    for row in title:
        created_at = datetime.datetime.strptime(row[-1], '%m/%d/%Y %H:%M')
        num = int(row[row_index])
        result.append([created_at, num])
        
    return result

In [11]:
# creating a frequency table with number of posts and comments or points by hour
def freq_table(list_name):    # list of lists in [creation time, number of comments/points] format
    
    posts_by_hour = dict()
    num_by_hour = dict()
    
    for entry in list_name:
        created_at = entry[0]
        num = entry[1]
        hour = created_at.strftime('%H')
    
        if hour not in posts_by_hour:
            posts_by_hour[hour] = 1
            num_by_hour[hour] = num
            
        else:
            posts_by_hour[hour] += 1
            num_by_hour[hour] += num
    
    return posts_by_hour, num_by_hour

In [12]:
result_list = iso_data(ask_posts, 4)

counts_by_hour, comments_by_hour = freq_table(result_list)
        
print('Number of posts for each hour:')
print(counts_by_hour)
print('Number of comments for each hour:')
print(comments_by_hour)

Number of posts for each hour:
{'02': 269, '01': 282, '22': 383, '21': 518, '19': 552, '17': 587, '15': 646, '14': 513, '13': 444, '11': 312, '10': 282, '09': 222, '07': 226, '03': 271, '23': 343, '20': 510, '16': 579, '08': 257, '00': 301, '18': 614, '12': 342, '04': 243, '06': 234, '05': 209}
Number of comments for each hour:
{'02': 2996, '01': 2089, '22': 3372, '21': 4500, '19': 3954, '17': 5547, '15': 18525, '14': 4972, '13': 7245, '11': 2797, '10': 3013, '09': 1477, '07': 1585, '03': 2154, '23': 2297, '20': 4462, '16': 4466, '08': 2362, '00': 2277, '18': 4877, '12': 4234, '04': 2360, '06': 1587, '05': 1838}


Now we have the number of posts and comments for each hour. With it we can find the average number of comments per post by hour.

### Counting the Average Number of Comments by Hour for Ask HN Posts

Let's find the average number of comments per **Ask HN** post by hour.

In [13]:
# counting average number of comments or points for ask hn, show hn or other posts by hour
def count_avg_by_hour(num_posts, num):    # num_posts: a dict with num of posts,
                                         # num: a dict with num of comments or points
    avg = list()
    
    for hour in num_posts:
        avg.append([hour, round(num[hour] / num_posts[hour], 2)])
    
    return avg

In [14]:
avg_by_hour = count_avg_by_hour(counts_by_hour, comments_by_hour)
    
print('Average number of comments per post by hour:')
print(avg_by_hour)

Average number of comments per post by hour:
[['02', 11.14], ['01', 7.41], ['22', 8.8], ['21', 8.69], ['19', 7.16], ['17', 9.45], ['15', 28.68], ['14', 9.69], ['13', 16.32], ['11', 8.96], ['10', 10.68], ['09', 6.65], ['07', 7.01], ['03', 7.95], ['23', 6.7], ['20', 8.75], ['16', 7.71], ['08', 9.19], ['00', 7.56], ['18', 7.94], ['12', 12.38], ['04', 9.71], ['06', 6.78], ['05', 8.79]]


We have average number of comments per post by hour but it is not sorted and not easy to read. We want to make this data clear. First step is to make this data sorted.

In [15]:
# swapping hour and avg num of comments or points and sort
def swap_and_sort(list_name):    # list of lists in [hour, avg num of comments/points] format
    
    swap = list()
    
    for hour in list_name:
        swap.append([hour[1], hour[0]])
        
    sort_list = sorted(swap, reverse = True)
    
    return sort_list

In [16]:
sorted_by_hour = swap_and_sort(avg_by_hour)

print('Average number of comments per post by hour from higher to lower:')
print(sorted_by_hour)

Average number of comments per post by hour from higher to lower:
[[28.68, '15'], [16.32, '13'], [12.38, '12'], [11.14, '02'], [10.68, '10'], [9.71, '04'], [9.69, '14'], [9.45, '17'], [9.19, '08'], [8.96, '11'], [8.8, '22'], [8.79, '05'], [8.75, '20'], [8.69, '21'], [7.95, '03'], [7.94, '18'], [7.71, '16'], [7.56, '00'], [7.41, '01'], [7.16, '19'], [7.01, '07'], [6.78, '06'], [6.7, '23'], [6.65, '09']]


Here is a sorted list with the average number of comments per post by hour. However it is still not easy to read. We are going to change it in the next section.

## Finding the Top 5 Hours for Ask Posts to Get the Most Comments

For our final step we are going to present our data in easy to read format.

In [17]:
print('The Top 5 Hours for Ask Posts Comments')

# for first 5 entries 
# changing hour format to hh:mm
# printing average number of comments per post by hour
for entry in sorted_by_hour[:4 + 1]:
    hour = datetime.datetime.strptime(entry[1], '%H')
    hour = hour.strftime('%H:%M')
    avg_comments = entry[0]
    
    print(f'{hour}: {avg_comments:.2f} average comments per post')

The Top 5 Hours for Ask Posts Comments
15:00: 28.68 average comments per post
13:00: 16.32 average comments per post
12:00: 12.38 average comments per post
02:00: 11.14 average comments per post
10:00: 10.68 average comments per post


As we can see, with 28.68 average comments per post, 15:00 leads the way. 13:00 is going next with 16.32 average number of comments per post. 12:00 have 12.38 average number of comments per post. 02:00 and 10:00 have 11.14 and 10.68 average number of comments per post respectively.

With this information we can say that post creation time has a big impact on the number of comments we receive. Best time for **Ask HN** post to get the most comments is between 12:00 and 15:00.

## Counting the Average Number of Points per Post on Ask HN and Show HN

Let's find out how many points both **Ask HN** and **Show HN** have in total.

In [18]:
total_ask_points = count_by_post(ask_posts, 3)
total_show_points = count_by_post(show_posts, 3)

print('There are {p:,} Ask HN points in total.'.format(p = total_ask_points))
print('There are {p:,} Show HN points in total.'.format(p = total_show_points))

There are 103,378 Ask HN points in total.
There are 150,781 Show HN points in total.


We can see that the **Show HN** posts have 150,781 comments in total and **Ask HN** have only 103,378 points in total. Now let's find the average number of points per post for both **Show HN** and **Ask HN** posts.

In [19]:
avg_ask_points = count_avg(ask_posts, total_ask_points)
avg_show_points = count_avg(show_posts, total_show_points)

print('Average number of points per \'Ask HN\' post:', round(avg_ask_points, 2))
print('Average number of points per \'Show HN\' post:', round(avg_show_points, 2))

Average number of points per 'Ask HN' post: 11.31
Average number of points per 'Show HN' post: 14.84


On average, for the **Show HN** we have 14.84 points per posts and for the **Ask HN** it's 11.31 points per post. This shows us that people are more willing to upvote **Show HN** posts than **Ask HN**. Based on this fact, for our next steps we will analyze only **Show HN** posts.

## Looking for the Average Number of Points by Hour for Show HN Posts

### Counting the Number of Posts and Points by Hour for Show HN Posts

The next step is to find out whether or not the creation time of the post affects the number of points received. We start by counting the number of posts and points by hour for **Show HN** posts. Doing so will help us to calculate the average number of points per post by hour.

In [20]:
result_list = iso_data(show_posts, 3)

counts_by_hour, points_by_hour = freq_table(result_list)

print('Number of posts for each hour:')
print(counts_by_hour)
print('Number of points for each hour:')
print(points_by_hour)

Number of posts for each hour:
{'00': 276, '23': 319, '20': 525, '19': 556, '18': 656, '16': 801, '14': 696, '10': 323, '09': 302, '08': 316, '06': 192, '03': 206, '21': 430, '17': 761, '15': 836, '11': 402, '07': 236, '04': 194, '13': 610, '12': 516, '01': 247, '22': 377, '02': 209, '05': 172}
Number of points for each hour:
{'00': 4291, '23': 5060, '20': 6948, '19': 8928, '18': 9935, '16': 11487, '14': 10503, '10': 4303, '09': 3762, '08': 4640, '06': 3071, '03': 2168, '21': 5990, '17': 10563, '15': 11657, '11': 7742, '07': 3303, '04': 2707, '13': 10381, '12': 10787, '01': 2931, '22': 5026, '02': 2764, '05': 1834}


Now we have the number of posts and points for each hour. With it, we can find the average number of points per post by hour.

### Counting the Average Number of Points by Hour for Show HN Posts

Let's find the average number of points per **Show HN** post by hour.

In [21]:
avg_by_hour = count_avg_by_hour(counts_by_hour, points_by_hour)

print(avg_by_hour)

[['00', 15.55], ['23', 15.86], ['20', 13.23], ['19', 16.06], ['18', 15.14], ['16', 14.34], ['14', 15.09], ['10', 13.32], ['09', 12.46], ['08', 14.68], ['06', 15.99], ['03', 10.52], ['21', 13.93], ['17', 13.88], ['15', 13.94], ['11', 19.26], ['07', 14.0], ['04', 13.95], ['13', 17.02], ['12', 20.91], ['01', 11.87], ['22', 13.33], ['02', 13.22], ['05', 10.66]]


We have an average number of points per post by hour, but it is not sorted and not easy to read. We want to make this data clear. First step is to make this data sorted.

In [22]:
sorted_by_hour = swap_and_sort(avg_by_hour)

print('Average number of points per post by hour from higher to lower:')
print(sorted_by_hour)

Average number of points per post by hour from higher to lower:
[[20.91, '12'], [19.26, '11'], [17.02, '13'], [16.06, '19'], [15.99, '06'], [15.86, '23'], [15.55, '00'], [15.14, '18'], [15.09, '14'], [14.68, '08'], [14.34, '16'], [14.0, '07'], [13.95, '04'], [13.94, '15'], [13.93, '21'], [13.88, '17'], [13.33, '22'], [13.32, '10'], [13.23, '20'], [13.22, '02'], [12.46, '09'], [11.87, '01'], [10.66, '05'], [10.52, '03']]


Here is a sorted list with the average number of points per post by hour. However, it is still not easy to read. We are going to change it in the next section.

## Finding the Top 5 Hours for Show Posts to Get the Most Points

For our final step we are going to present our data in an easy to read format.

In [23]:
print('The Top 5 Hours for Show Posts Points')

# for first 5 entries 
# changing hour format to hh:mm
# printing average number of points per post by hour
for entry in sorted_by_hour[:4 + 1]:
    hour = datetime.datetime.strptime(entry[1], '%H')
    hour = hour.strftime('%H:%M')
    avg_points = entry[0]
    
    print(f'{hour}: {avg_points:.2f} average points per post')

The Top 5 Hours for Show Posts Points
12:00: 20.91 average points per post
11:00: 19.26 average points per post
13:00: 17.02 average points per post
19:00: 16.06 average points per post
06:00: 15.99 average points per post


As we can see, with 20.91 average points per post, 12:00 leads the way. 11:00 is going next with 19.26 average number of points per post. 13:00 have 17.02 average number of comments per post. 19:00 and 06:00 have 16.06 and 15.99 average number of points per post respectively.

With this information, we can say that post creation time has a really small impact on the number of points we receive. But the most number of points for **Show HN** post you can achieve in between 11:00 and 13:00.

## Comparing of Show HN and Ask HN posts with other posts. Who Gets More Comments and Points?

Now, let's compare our findings with other posts.

In [24]:
total_other_comments = count_by_post(other_posts, 4)
total_other_points = count_by_post(other_posts, 3)

print('There are {c:,} Ask HN comments in total'.format(c = total_ask_comments))
print('There are {c:,} Show HN comments in total'.format(c = total_show_comments))
print('There are {c:,} other posts comments in total'.format(c = total_other_comments))
print('\n')
print('There are {p:,} Ask HN points in total.'.format(p = total_ask_points))
print('There are {p:,} Show HN points in total.'.format(p = total_show_points))
print('There are {p:,} other posts points in total.'.format(p = total_other_points))

There are 94,986 Ask HN comments in total
There are 49,633 Show HN comments in total
There are 1,768,142 other posts comments in total


There are 103,378 Ask HN points in total.
There are 150,781 Show HN points in total.
There are 4,150,049 other posts points in total.


For raw data, it's obvious that other posts have more points and comments in total than both **Show HN** and **Ask HN** posts. But, let's find out if the average value is greater too.

In [25]:
avg_other_comments = count_avg(other_posts, total_other_comments)
avg_other_points = count_avg(other_posts, total_other_points)

print('Average number of comments per \'Ask HN\' post:', round(avg_ask_comments, 2))
print('Average number of comments per \'Show HN\' post:', round(avg_show_comments, 2))
print('Average number of comments for other posts per post', round(avg_other_comments, 2))
print('\n')
print('Average number of points per \'Ask HN\' post:', round(avg_ask_points, 2))
print('Average number of points per \'Show HN\' post:', round(avg_show_points, 2))
print('Average number of points for other posts per post', round(avg_other_points, 2))

Average number of comments per 'Ask HN' post: 10.39
Average number of comments per 'Show HN' post: 4.89
Average number of comments for other posts per post 6.46


Average number of points per 'Ask HN' post: 11.31
Average number of points per 'Show HN' post: 14.84
Average number of points for other posts per post 15.16


As we can see, other posts have 6.46 comments per post on average. It's greater than **Show HN** posts with 4.89 comments per post, but lower than **Ask HN** posts with 10.39 comments per post. As for points, other posts have 15.16 points per post on average. This is greater than both **Ask HN** and **Show HN** posts with 11.31 and 14.84 points per post on average respectively.

We can say now, that other types of posts are more popular in terms of points. However, people are more likely to leave a comment on **Ask HN** post than on other kinds of posts.

## Conclusion

The goal of our project was as follows. To understand which **Ask HN** or **Show HN** posts get more comments on average and whether time of the day affects this. To do this, we first calculated the average number of comments for each type of posts. Then we calculated the average number of comments **Ask HN** posts receive by hour created to see if it affects the average number of comments.

In the end, we learned that **Ask HN** posts gain a lot more comments on average. Time is also greatly affects the average number of comments per post. The best hours for **Ask HN** posts is between 12:00 and 15:00.

As for our extra research, we learned the following. **Show HN** posts gather more points on average than **Ask HN** posts. But in this case, time of the day have a small impact on the number of points received. However, the hours with most number of points is between 11:00 and 13:00.